In [4]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
%load_ext Cython

In [ ]:
%%cython
def function():
    cdef int a=2

In [11]:
class particleClass():
    nMade=0
    def __init__(self):
        self.pos=np.zeros(3)
        self.vel=np.zeros(3)
        self.m=0
        self.index=particleClass.nMade
        particleClass.nMade+=1

In [26]:
def runSim(particles,boxSize,dt,endTime,nPlots): #simple leapfrog integration
    nTimestep=int(endTime/dt)
    for i in range(nTimestep):
        for p in particles: #to do leapfrog integration need to update all positions
            p.pos=(p.pos+dt*p.vel) % boxSize
        for p in particles: #then compute new accelerations on this evolved systems
            acc=findAcc(particles,p)
            p.vel=p.vel+dt*acc
        if (i%int(nTimestep/nPlots)==0): #periodically plots the state of the box
            plotSim(particles,i*dt,i/int(nTimestep/nPlots))

def findAcc(particles,thisParticle):
    pos=thisParticle.pos
    index=thisParticle.index
    acc=np.zeros(3)
    for p in particles:
        e_r=(pos-p.pos) #vector from loop particle to this particle
        for i in range(3): #checks periodic boundary conditions
            if (e_r[i] > boxSize/2):
                e_r[i]-=boxSize
            elif (e_r[i] < -boxSize/2):
                e_r[i]+=boxSize
        r=np.linalg.norm(e_r)+0.1 #0.1 is gravitational softening length
        e_r=e_r/r # normalises to make into unit vector
        if (p.index == index): # the particle should feel no force from itself
            continue
        acc+=(-p.m/(r**2))*e_r # == GM/r^3 _r_ in units where G=1
    return acc

def initSim(nParticles,boxSize):
    pMass=1 #mass of each particle
    particleClass.nMade=0
    particles=[]
    for i in range(nParticles):
        newP=particleClass()
        newP.pos=boxSize*np.random.random(3)
        newP.vel=(boxSize/100)*(1-2*np.random.random(3)) # units be damned...
        newP.m=pMass
        particles.append(newP)
    return particles

def plotSim(particles,time,whichPlot):
    fig=plt.subplot(grid[int(whichPlot),0])
    fig.set_xlim(0,boxSize)
    fig.set_ylim(0,boxSize)
    fig.set_xlabel("x")
    fig.set_ylabel("y")
    fig.set_title('t='+str(time))
    for p in particles:
        fig.scatter(p.pos[0],p.pos[1],color=cmap(p.index/particleClass.nMade),s=1)
        #fig.quiver(p.pos[0],p.pos[1],p.vel[0],p.vel[1])#,scale=0.01*np.linalg.norm(p.vel))

In [28]:
nParticles=40
boxSize=200 # not periodic!
dt=0.1 # in units where G=1
endTime=20
#in these units it takes roughly t=80 for two particles of mass 1 
#to cross corner to corner of boxSize=10 and come to rest

nPlots=4
plots=plt.figure(figsize=(6,nPlots*6))
grid=matplotlib.gridspec.GridSpec(nPlots,1)
cmap=matplotlib.cm.get_cmap('inferno')

particles=initSim(nParticles,boxSize)
runSim(particles,boxSize,dt,endTime,nPlots)

plt.show()